In [1]:
import csv
import numpy as np
import os
import pandas as pd
import requests
import time

In [4]:
asso = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/7_new_association/bi_random_walk_Cdataset_association.tsv")

In [62]:
# read the mimTitles file
mimTitles = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/6_new_disease/mimTitles.txt", skiprows = 2)
mimTitles = mimTitles.rename(columns = {"# Prefix" : "Prefix","Mim Number" : "Mim_Number", "Preferred Title; symbol" : "Preferred_Title"})
print len(mimTitles)
mimTitles.head(1)

20516


,Prefix,Mim_Number,Preferred_Title,Alternative Title(s); symbol(s),Included Title(s); symbols
0,NaN,100050,"AARSKOG SYNDROME, AUTOSOMAL DOMINANT",NaN,NaN


In [37]:
# read the morbidmap file
morbid = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/6_new_disease/morbidmap_txt.txt")
print len(morbid)
morbid.head(1)

7257


,Phenotype,Gene_Symbols,MIM_Number,Cyto_Location
0,"17,20-lyase deficiency, isolated, 202110 (3)","CYP17A1, CYP17, P450C17",609300.0,10q24.32


In [38]:
print len(asso)
asso.head(1)

2532


,disease_id,drug_id
0,D102100,DB00555


In [45]:
di_mimNum = asso['disease_id'].drop_duplicates().str[1:]

In [55]:
int_di_mimNum = list()
for i in di_mimNum:
    int_di_mimNum.append (int(i))

In [64]:
df_disease = mimTitles[mimTitles['Mim_Number'].isin(int_di_mimNum)]

In [65]:
print len(df_disease)
df_disease.head(1)

409


,Prefix,Mim_Number,Preferred_Title,Alternative Title(s); symbol(s),Included Title(s); symbols
39,Percent,102100,"ACROMEGALOID CHANGES, CUTIS VERTICIS GYRATA, A...",ROSENTHAL-KLOEPFER SYNDROME,NaN


In [72]:
pair= pd.DataFrame()
for index, row in df_disease.iterrows():
    mim_number = row['Mim_Number']
    title = row['Preferred_Title']
    temp_df = pd.DataFrame([[mim_number, title]], columns = ['Mim_Number','Preferred_Title'])
    pair = pair.append(temp_df)

In [90]:
print len(pair)
pair.head(1)

409


,Mim_Number,Preferred_Title
0,102100,"ACROMEGALOID CHANGES, CUTIS VERTICIS GYRATA, A..."


In [169]:
di_phen = pd.DataFrame()
result = pd.DataFrame()
temp = pd.DataFrame()
cnt = 0
for index, row in pair.iterrows():
    di_mim_number = row['Mim_Number']
    found = morbid[morbid['Phenotype'].str.contains(str(di_mim_number))]
    if found.empty :
        continue
    else:
        cnt = cnt + 1
        phe_mim_number = found['MIM_Number']
        #temp_df = pd.DataFrame(data = [[di_mim_number, phe_mim_number]], columns = ['disease', 'phenotype'])
        temp_df = pd.DataFrame({'disease' : di_mim_number, 'phenotype' : phe_mim_number})
        temp_df2 = pd.DataFrame({'disease' : di_mim_number, 'disease_name' : row['Preferred_Title'], 'phenotype' : phe_mim_number, 'phenotype_title' : found['Phenotype']})
        di_phen = di_phen.append(temp_df)
        result = result.append(found)
        temp = temp.append(temp_df2)

In [170]:
temp.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/6_new_disease/test.tsv", sep='\t', index=False)

In [124]:
cnt

223

In [125]:
print len(result)
result.head()

587


,Phenotype,Gene_Symbols,MIM_Number,Cyto_Location
2698,"Hajdu-Cheney syndrome, 102500 (3)","NOTCH2, AGS2, HJCYS",600275.0,1p12
462,"ADULT syndrome, 103285 (3)","TP63, TP73L, KET, EEC3, SHFM4, LMS, RHS, OFC8",603273.0,3q28
6159,"{Alcohol dependence, protection against}, 1037...","ADH1B, ADH2",103720.0,4q23
6160,"{Alcohol dependence, protection against}, 1037...","ADH1C, ADH3",103730.0,4q23
6161,"{Alcohol dependence, susceptibility to}, 10378...",GABRA2,137140.0,4p12


In [126]:
print len(di_phen)
di_phen.head()

587


,disease,phenotype
2698,102500,600275.0
462,103285,603273.0
6159,103780,103720.0
6160,103780,103730.0
6161,103780,137140.0


In [127]:
print len(di_phen.drop_duplicates(subset = 'disease'))
print len(di_phen.drop_duplicates(subset = 'phenotype'))

223
452


In [133]:
left = pair[~pair['Mim_Number'].isin(di_phen['disease'].drop_duplicates())]

In [141]:
left.to_csv("/home/share/aschoi/data/Drug_Repositioning/6_new_disease/temp2.tsv", sep="\t", index=False)

In [135]:
result1 = result

In [164]:
cnt = 0 
result2 = pd.DataFrame()
temp = pd.DataFrame()
for index, row in left.iterrows():
    di_mim_number = row['Mim_Number']
    title = row['Preferred_Title'].split(', ')[0]
    found = morbid[morbid['Phenotype'].str.contains(title, case=False)]
    if found.empty :
        continue
    else:
        cnt = cnt + 1
        phe_mim_number = found['MIM_Number']
        #temp_df = pd.DataFrame(data = [[di_mim_number, phe_mim_number]], columns = ['disease', 'phenotype'])
        #temp_df = pd.DataFrame({'disease' : di_mim_number, 'phenotype' : phe_mim_number})
        temp_df = pd.DataFrame({'disease' : di_mim_number, 'disease_name' : row['Preferred_Title'], 'phenotype' : phe_mim_number, 'phenotype_title' : found['Phenotype']})
        #di_phen = di_phen.append(temp_df)
        temp = temp.append(temp_df)
        result2 = result2.append(found)

In [161]:
title

'CHROMOSOME 15q11-q13 DUPLICATION SYNDROME'

In [165]:
cnt

92

In [166]:
print len(result2)
result2.head()

2475


,Phenotype,Gene_Symbols,MIM_Number,Cyto_Location
7056,{Restless legs syndrome 1} (2),RLS1,102300.0,12q12-q21
7057,{Restless legs syndrome 2} (2),RLS2,608831.0,14q13-q21
7058,{Restless legs syndrome 3} (2),RLS3,610438.0,9p24-p22
7059,{Restless legs syndrome 4} (2),RLS4,610439.0,2q33
7060,{Restless legs syndrome 5} (2),RLS5,611242.0,20p13


In [168]:
temp.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/6_new_disease/test.tsv", sep='\t', index=False)

In [167]:
print len(temp)
temp

2475


,disease,disease_name,phenotype,phenotype_title
7056,102300,"RESTLESS LEGS SYNDROME, SUSCEPTIBILITY TO, 1; ...",102300.0,{Restless legs syndrome 1} (2)
7057,102300,"RESTLESS LEGS SYNDROME, SUSCEPTIBILITY TO, 1; ...",608831.0,{Restless legs syndrome 2} (2)
7058,102300,"RESTLESS LEGS SYNDROME, SUSCEPTIBILITY TO, 1; ...",610438.0,{Restless legs syndrome 3} (2)
7059,102300,"RESTLESS LEGS SYNDROME, SUSCEPTIBILITY TO, 1; ...",610439.0,{Restless legs syndrome 4} (2)
7060,102300,"RESTLESS LEGS SYNDROME, SUSCEPTIBILITY TO, 1; ...",611242.0,{Restless legs syndrome 5} (2)
7061,102300,"RESTLESS LEGS SYNDROME, SUSCEPTIBILITY TO, 1; ...",611185.0,{Restless legs syndrome 6} (2)
7062,102300,"RESTLESS LEGS SYNDROME, SUSCEPTIBILITY TO, 1; ...",612853.0,{Restless legs syndrome 7} (2)
7063,102300,"RESTLESS LEGS SYNDROME, SUSCEPTIBILITY TO, 1; ...",615197.0,{Restless legs syndrome 8} (2)
44,104130,"ALOPECIA, PSYCHOMOTOR EPILEPSY, PYORRHEA, AND ...",612074.0,"?Alopecia, neurologic defects, and endocrinopa..."
589,104130,"ALOPECIA, PSYCHOMOTOR EPILEPSY, PYORRHEA, AND ...",104000.0,Alopecia areata 1 (2)


In [172]:
found = morbid[morbid['Phenotype'].str.contains('ALOPECIA', case=False)]
found

,Phenotype,Gene_Symbols,MIM_Number,Cyto_Location
44,"?Alopecia, neurologic defects, and endocrinopa...","RBM28, ANES",612074.0,7q32.1
589,Alopecia areata 1 (2),AA1,104000.0,18p11.3-p11.2
590,Alopecia areata 2 (2),AA2,610753.0,16q11-q22
591,"Alopecia universalis, 203655 (3)","HR, AU, MUHH1, HYPT4",602302.0,8p21.3
592,"Alopecia, androgenetic, 1 (2)","AFA1, MPB",109200.0,3q26
593,"Alopecia, androgenetic, 2 (2)",AGA2,300710.0,Xq11-q12
594,"Alopecia, androgenetic, 3 (2)",AGA3,612421.0,20p11.22
595,Alopecia-mental retardation syndrome 1 (2),APMR1,203650.0,3q26.3-q27.3
596,Alopecia-mental retardation syndrome 2 (2),APMR2,610422.0,3q26.2-q26.31
597,Alopecia-mental retardation syndrome 3 (2),APMR3,613930.0,18q11.2-q12.2


In [173]:
len(found)

15